In [ ]:
### https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py

''' Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

from Sonnet_Set import Sonnet_Set
from Sonnet_Set import Sequence_Type
from Sonnet_Set import Element_Type

sonnet_set = Sonnet_Set("data/shakespeare.txt")
sonnets = open("data/shakespeare.txt")
fulltext =  sonnets.readlines()

text =  "\n".join(fulltext)
#path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
#with io.open(path, encoding='utf-8') as f:
#    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Sonnet 99 is not 14 lines, skipping
Sonnet 126 is not 14 lines, skipping
corpus length: 100643
total chars: 71
nb sequences: 33535
Vectorization...
Build model...
Epoch 1/60
33535/33535 [==============================] - 35s 1ms/step - loss: 2.3649

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "make faults, and even I in this,

Author"
make faults, and even I in this,

Authore that the me the make that stall,

  But that that me that the that that fall,

  That that that meath that that that meart,

  But that pain that that will that meart,

  That the mint that that whealt that stall stall stall stall stall theing,

  The the pall that that stall that dees,

Wheng thealt that that that that stall stall pane,

When will meall my the fall stall stain,

  But the fall 
----- diversity: 0.5
----- Generating with seed: "make faults, and even I in this,

Author"
make faults, and even I in this,

Authore theave thate hath seate that peall,

The  fate

In [10]:
allsonnets = "\n".join(fulltext)

In [11]:
allsonnets

"                   1\n\nFrom fairest creatures we desire increase,\n\nThat thereby beauty's rose might never die,\n\nBut as the riper should by time decease,\n\nHis tender heir might bear his memory:\n\nBut thou contracted to thine own bright eyes,\n\nFeed'st thy light's flame with self-substantial fuel,\n\nMaking a famine where abundance lies,\n\nThy self thy foe, to thy sweet self too cruel:\n\nThou that art now the world's fresh ornament,\n\nAnd only herald to the gaudy spring,\n\nWithin thine own bud buriest thy content,\n\nAnd tender churl mak'st waste in niggarding:\n\n  Pity the world, or else this glutton be,\n\n  To eat the world's due, by the grave and thee.\n\n\n\n\n\n                   2\n\nWhen forty winters shall besiege thy brow,\n\nAnd dig deep trenches in thy beauty's field,\n\nThy youth's proud livery so gazed on now,\n\nWill be a tattered weed of small worth held:\n\nThen being asked, where all thy beauty lies,\n\nWhere all the treasure of thy lusty days;\n\nTo say 